# BRNet training

# 1. SSUB

In [1]:
#https://github.com/wolverton-research-group/qmpy/blob/master/qmpy/data/thermodata/ssub.dat
from jarvis.db.figshare import  get_request_data
dat = get_request_data(js_tag="ssub.json",url="https://figshare.com/ndownloader/files/40084921")

Loading the zipfile...
Loading completed.


In [2]:
import pandas as pd
df = pd.DataFrame(dat)

In [3]:
df

,id,formula_energy,formula
0,465,-2.841796e+00,CaO4W
1,1197,1.766051e-09,Ho
2,1759,-3.555857e-01,PbTe
3,1454,-9.540103e-01,MoS2
4,218,-6.564038e-02,BeH2
...,...,...,...
1722,1482,-1.892842e+00,NZr
1723,773,-2.404511e+00,CrLiO2
1724,251,-4.946426e-01,BiNa3
1725,328,-7.213532e-01,Br4Ge


In [4]:
import json,zipfile
import numpy as np
path = "../../benchmarks/AI/SinglePropertyPrediction/ssub_formula_energy.json.zip"
js_tag = "ssub_formula_energy.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))

In [5]:
id_data.keys()

dict_keys(['train', 'test'])

In [6]:
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')

In [7]:
print(len(train_ids),len(test_ids),len(train_ids)+len(test_ids))

1381 345 1726


In [8]:
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

In [9]:
len(train_df),  len(test_df)

(1381, 345)

In [10]:
%%time
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['formula_energy'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['formula_energy'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')


CPU times: user 194 ms, sys: 18 ms, total: 212 ms
Wall time: 209 ms


In [11]:
#Split train into train-val split
n_train = int(len(X_train)*0.8)
n_val = len(X_train)-n_train
n_train,n_val,X_train.shape
X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_val = X_train[-n_val:]
y_val = y_train[-n_val:]

In [12]:
X_train.shape,X_test.shape

((1104, 103), (345, 103))

In [13]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import add
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error

2023-05-23 12:49:54.922242: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 12:49:55.685838: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-23 12:49:55.685944: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-23 12:49:55.685954: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [14]:
#Network from "BRNet: Branched Residual Network for Fast and Accurate Predictive Modeling of Materials Properties"
#Link to paper: https://epubs.siam.org/doi/10.1137/1.9781611977172.39
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = LeakyReLU()(layer_1)

fcc_1 = Dense(1024)(in_layer)
gsk_1 = add([fcc_1, layer_1])

layer_2 = Dense(1024)(gsk_1)
layer_2 = LeakyReLU()(layer_2)

gsk_2 = add([gsk_1, layer_2])


rayer_1 = Dense(1024)(in_layer)
rayer_1 = LeakyReLU()(rayer_1)

rcc_1 = Dense(1024)(in_layer)
rsk_1 = add([rcc_1, rayer_1])

rayer_2 = Dense(1024)(rsk_1)
rayer_2 = LeakyReLU()(rayer_2)

rsk_2 = add([rsk_1, rayer_2])


mayer_1 = add([gsk_2, rsk_2])


layer_5 = Dense(512)(mayer_1)
layer_5 = LeakyReLU()(layer_5)

mcc_5 = Dense(512)(mayer_1)
msk_5 = add([mcc_5, layer_5])

layer_6 = Dense(512)(msk_5)
layer_6 = LeakyReLU()(layer_6)

msk_6 = add([msk_5, layer_6])

layer_7 = Dense(512)(msk_6)
layer_7 = LeakyReLU()(layer_7)

msk_7 = add([msk_6, layer_7])

layer_8 = Dense(256)(msk_7)
layer_8 = LeakyReLU()(layer_8)

mcc_8 = Dense(256)(msk_7)
msk_8 = add([mcc_8, layer_8])

layer_9 = Dense(256)(msk_8)
layer_9 = LeakyReLU()(layer_9)

msk_9 = add([msk_8, layer_9])

layer_10 = Dense(256)(msk_9)
layer_10 = LeakyReLU()(layer_10)

msk_10 = add([msk_9, layer_10])

layer_11 = Dense(128)(msk_10)
layer_11 = LeakyReLU()(layer_11)

mcc_11 = Dense(128)(msk_10)
msk_11 = add([mcc_11, layer_11])

layer_12 = Dense(128)(msk_11)
layer_12 = LeakyReLU()(layer_12)

msk_12 = add([msk_11, layer_12])

layer_13 = Dense(128)(msk_12)
layer_13 = LeakyReLU()(layer_13)

msk_13 = add([msk_12, layer_13])

layer_14 = Dense(64)(msk_13)
layer_14 = LeakyReLU()(layer_14)

mcc_14 = Dense(64)(msk_13)
msk_14 = add([mcc_14, layer_14])

layer_15 = Dense(64)(msk_14)
layer_15 = LeakyReLU()(layer_15)

msk_15 = add([msk_14, layer_15])

layer_16 = Dense(32)(msk_15)
layer_16 = LeakyReLU()(layer_16)

mcc_16 = Dense(32)(msk_15)
msk_16 = add([mcc_16, layer_16])

out_layer = Dense(1)(msk_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


2023-05-23 12:49:56.497874: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 12:49:57.886381: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14611 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
2023-05-23 12:49:57.887577: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14611 MB memory:  -> device: 1, name: Tesla V100-PCIE-16GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


In [15]:
%%time
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000


2023-05-23 12:50:01.594054: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f0904040030 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-23 12:50:01.594145: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla V100-PCIE-16GB, Compute Capability 7.0
2023-05-23 12:50:01.594171: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): Tesla V100-PCIE-16GB, Compute Capability 7.0
2023-05-23 12:50:01.608060: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-23 12:50:02.042451: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


35/35 - 9s - loss: 4.9592 - mean_absolute_error: 4.9592 - val_loss: 1.3273 - val_mean_absolute_error: 1.3273 - 9s/epoch - 250ms/step
Epoch 2/3000
35/35 - 1s - loss: 0.9419 - mean_absolute_error: 0.9419 - val_loss: 0.7682 - val_mean_absolute_error: 0.7682 - 562ms/epoch - 16ms/step
Epoch 3/3000
35/35 - 1s - loss: 0.7590 - mean_absolute_error: 0.7590 - val_loss: 0.6922 - val_mean_absolute_error: 0.6922 - 588ms/epoch - 17ms/step
Epoch 4/3000
35/35 - 1s - loss: 0.5814 - mean_absolute_error: 0.5814 - val_loss: 0.4039 - val_mean_absolute_error: 0.4039 - 583ms/epoch - 17ms/step
Epoch 5/3000
35/35 - 1s - loss: 0.4325 - mean_absolute_error: 0.4325 - val_loss: 0.5270 - val_mean_absolute_error: 0.5270 - 557ms/epoch - 16ms/step
Epoch 6/3000
35/35 - 1s - loss: 0.4728 - mean_absolute_error: 0.4728 - val_loss: 0.3430 - val_mean_absolute_error: 0.3430 - 558ms/epoch - 16ms/step
Epoch 7/3000
35/35 - 1s - loss: 0.4092 - mean_absolute_error: 0.4092 - val_loss: 0.2783 - val_mean_absolute_error: 0.2783 - 588

In [16]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

11/11 [==============================] - 0s 3ms/step
0.11429334769435832


In [17]:
# pred=model.predict(X_test)
# print(mean_absolute_error(y_test,pred))
# 0.17889024475861343

In [18]:
import os
f=open('AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k[0])+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv.zip AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv'
os.system(cmd)


updating: AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv (deflated 52%)


0

# 2. SuperCon

In [19]:
from jarvis.db.figshare import  get_request_data
import pandas as pd
dat = get_request_data(js_tag="supercon_chem.json",url="https://figshare.com/ndownloader/files/40719260")
#http://supercon.nims.go.jp/index_en.html
#https://github.com/vstanev1/Supercon

df = pd.DataFrame(dat)

Loading the zipfile...
Loading completed.


In [20]:
df

,id,Tc,formula
0,1524,0.00,Tb1Th3.4Ru0.6B4
1,14309,0.00,B2Th1
2,6607,1.09,Ni2.5Cu1.5Zr8
3,4010,24.60,Pr0.91La1Ce0.09Cu1O4
4,14511,55.00,Pb0.5Ca1Sr2Y0.5Cu2O6.97
...,...,...,...
16409,442,0.00,As1Pd7
16410,14942,0.00,La1.5Gd1.5Ca1Ba2Sr1Cu6.3In0.7O16.62
16411,3978,0.00,N0.98Pr1
16412,1078,32.00,Sm0.1La1.7Sr0.2Cu1O4


In [21]:
import json,zipfile
import numpy as np
path = "../../benchmarks/AI/SinglePropertyPrediction/supercon_chem_Tc.json.zip"
js_tag = "supercon_chem_Tc.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

In [22]:
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['Tc'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['Tc'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')
#Split train into train-val split
n_train = int(len(X_train)*0.8)
n_val = len(X_train)-n_train
n_train,n_val,X_train.shape
X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_val = X_train[-n_val:]
y_val = y_train[-n_val:]

In [23]:
X_train.shape,X_test.shape

((10504, 103), (3283, 103))

In [24]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Network from "BRNet: Branched Residual Network for Fast and Accurate Predictive Modeling of Materials Properties"
#Link to paper: https://epubs.siam.org/doi/10.1137/1.9781611977172.39
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = LeakyReLU()(layer_1)

fcc_1 = Dense(1024)(in_layer)
gsk_1 = add([fcc_1, layer_1])

layer_2 = Dense(1024)(gsk_1)
layer_2 = LeakyReLU()(layer_2)

gsk_2 = add([gsk_1, layer_2])


rayer_1 = Dense(1024)(in_layer)
rayer_1 = LeakyReLU()(rayer_1)

rcc_1 = Dense(1024)(in_layer)
rsk_1 = add([rcc_1, rayer_1])

rayer_2 = Dense(1024)(rsk_1)
rayer_2 = LeakyReLU()(rayer_2)

rsk_2 = add([rsk_1, rayer_2])


mayer_1 = add([gsk_2, rsk_2])


layer_5 = Dense(512)(mayer_1)
layer_5 = LeakyReLU()(layer_5)

mcc_5 = Dense(512)(mayer_1)
msk_5 = add([mcc_5, layer_5])

layer_6 = Dense(512)(msk_5)
layer_6 = LeakyReLU()(layer_6)

msk_6 = add([msk_5, layer_6])

layer_7 = Dense(512)(msk_6)
layer_7 = LeakyReLU()(layer_7)

msk_7 = add([msk_6, layer_7])

layer_8 = Dense(256)(msk_7)
layer_8 = LeakyReLU()(layer_8)

mcc_8 = Dense(256)(msk_7)
msk_8 = add([mcc_8, layer_8])

layer_9 = Dense(256)(msk_8)
layer_9 = LeakyReLU()(layer_9)

msk_9 = add([msk_8, layer_9])

layer_10 = Dense(256)(msk_9)
layer_10 = LeakyReLU()(layer_10)

msk_10 = add([msk_9, layer_10])

layer_11 = Dense(128)(msk_10)
layer_11 = LeakyReLU()(layer_11)

mcc_11 = Dense(128)(msk_10)
msk_11 = add([mcc_11, layer_11])

layer_12 = Dense(128)(msk_11)
layer_12 = LeakyReLU()(layer_12)

msk_12 = add([msk_11, layer_12])

layer_13 = Dense(128)(msk_12)
layer_13 = LeakyReLU()(layer_13)

msk_13 = add([msk_12, layer_13])

layer_14 = Dense(64)(msk_13)
layer_14 = LeakyReLU()(layer_14)

mcc_14 = Dense(64)(msk_13)
msk_14 = add([mcc_14, layer_14])

layer_15 = Dense(64)(msk_14)
layer_15 = LeakyReLU()(layer_15)

msk_15 = add([msk_14, layer_15])

layer_16 = Dense(32)(msk_15)
layer_16 = LeakyReLU()(layer_16)

mcc_16 = Dense(32)(msk_15)
msk_16 = add([mcc_16, layer_16])

out_layer = Dense(1)(msk_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


In [25]:
%%time
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000
329/329 - 12s - loss: 13.1211 - mean_absolute_error: 13.1211 - val_loss: 13.4269 - val_mean_absolute_error: 13.4269 - 12s/epoch - 35ms/step
Epoch 2/3000
329/329 - 5s - loss: 11.3546 - mean_absolute_error: 11.3546 - val_loss: 11.5217 - val_mean_absolute_error: 11.5217 - 5s/epoch - 15ms/step
Epoch 3/3000
329/329 - 5s - loss: 10.6732 - mean_absolute_error: 10.6732 - val_loss: 9.9736 - val_mean_absolute_error: 9.9736 - 5s/epoch - 15ms/step
Epoch 4/3000
329/329 - 5s - loss: 10.3118 - mean_absolute_error: 10.3118 - val_loss: 12.6605 - val_mean_absolute_error: 12.6605 - 5s/epoch - 15ms/step
Epoch 5/3000
329/329 - 5s - loss: 10.0982 - mean_absolute_error: 10.0982 - val_loss: 9.1067 - val_mean_absolute_error: 9.1067 - 5s/epoch - 15ms/step
Epoch 6/3000
329/329 - 5s - loss: 9.5838 - mean_absolute_error: 9.5838 - val_loss: 9.1785 - val_mean_absolute_error: 9.1785 - 5s/epoch - 15ms/step
Epoch 7/3000
329/329 - 5s - loss: 9.5791 - mean_absolute_error: 9.5791 - val_loss: 9.1398 - val_mean

In [26]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

103/103 [==============================] - 1s 4ms/step
5.437043767822163


In [27]:
import os
f=open('AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k[0])+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv.zip AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv'
os.system(cmd)


updating: AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv (deflated 53%)


0

# 3. Magnetic2DChem

In [28]:
import json,zipfile
import numpy as np
from jarvis.db.figshare import  get_request_data
import pandas as pd
dat = get_request_data(js_tag="mag2d_chem.json",url="https://figshare.com/ndownloader/files/40720004")
#http://supercon.nims.go.jp/index_en.html
#https://github.com/vstanev1/Supercon

df = pd.DataFrame(dat)
path = "../../benchmarks/AI/SinglePropertyPrediction/mag2d_chem_magnetic_moment.json.zip"
js_tag = "mag2d_chem_magnetic_moment.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

Loading the zipfile...
Loading completed.


In [29]:
df

,id,magnetic_moment,formula
0,49,5.0780,V1Cr1Ge2Te6
1,4,3.1560,Cr1Co1Si2Te6
2,212,3.0161,Ta1Cr1Sn1Ge1Te6
3,153,2.7459,Cr1Sn1Te6P1Au1
4,46,2.2800,Ti1Cr1Si2Te6
...,...,...,...
221,57,2.9670,Y1Cr1Si1Te6P1
222,97,5.6479,Cr1Ge2Te6W1
223,60,2.6981,Ta1Cr1In1Te6Pb1
224,152,5.0129,Cr1Sn1Te6Mo1P1


In [30]:
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['magnetic_moment'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['magnetic_moment'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')
#Split train into train-val split
n_train = int(len(X_train)*0.8)
n_val = len(X_train)-n_train
n_train,n_val,X_train.shape
X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_val = X_train[-n_val:]
y_val = y_train[-n_val:]

In [31]:
X_train.shape,X_test.shape

((144, 103), (46, 103))

In [32]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error

#Network from "BRNet: Branched Residual Network for Fast and Accurate Predictive Modeling of Materials Properties"
#Link to paper: https://epubs.siam.org/doi/10.1137/1.9781611977172.39
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = LeakyReLU()(layer_1)

fcc_1 = Dense(1024)(in_layer)
gsk_1 = add([fcc_1, layer_1])

layer_2 = Dense(1024)(gsk_1)
layer_2 = LeakyReLU()(layer_2)

gsk_2 = add([gsk_1, layer_2])


rayer_1 = Dense(1024)(in_layer)
rayer_1 = LeakyReLU()(rayer_1)

rcc_1 = Dense(1024)(in_layer)
rsk_1 = add([rcc_1, rayer_1])

rayer_2 = Dense(1024)(rsk_1)
rayer_2 = LeakyReLU()(rayer_2)

rsk_2 = add([rsk_1, rayer_2])


mayer_1 = add([gsk_2, rsk_2])


layer_5 = Dense(512)(mayer_1)
layer_5 = LeakyReLU()(layer_5)

mcc_5 = Dense(512)(mayer_1)
msk_5 = add([mcc_5, layer_5])

layer_6 = Dense(512)(msk_5)
layer_6 = LeakyReLU()(layer_6)

msk_6 = add([msk_5, layer_6])

layer_7 = Dense(512)(msk_6)
layer_7 = LeakyReLU()(layer_7)

msk_7 = add([msk_6, layer_7])

layer_8 = Dense(256)(msk_7)
layer_8 = LeakyReLU()(layer_8)

mcc_8 = Dense(256)(msk_7)
msk_8 = add([mcc_8, layer_8])

layer_9 = Dense(256)(msk_8)
layer_9 = LeakyReLU()(layer_9)

msk_9 = add([msk_8, layer_9])

layer_10 = Dense(256)(msk_9)
layer_10 = LeakyReLU()(layer_10)

msk_10 = add([msk_9, layer_10])

layer_11 = Dense(128)(msk_10)
layer_11 = LeakyReLU()(layer_11)

mcc_11 = Dense(128)(msk_10)
msk_11 = add([mcc_11, layer_11])

layer_12 = Dense(128)(msk_11)
layer_12 = LeakyReLU()(layer_12)

msk_12 = add([msk_11, layer_12])

layer_13 = Dense(128)(msk_12)
layer_13 = LeakyReLU()(layer_13)

msk_13 = add([msk_12, layer_13])

layer_14 = Dense(64)(msk_13)
layer_14 = LeakyReLU()(layer_14)

mcc_14 = Dense(64)(msk_13)
msk_14 = add([mcc_14, layer_14])

layer_15 = Dense(64)(msk_14)
layer_15 = LeakyReLU()(layer_15)

msk_15 = add([msk_14, layer_15])

layer_16 = Dense(32)(msk_15)
layer_16 = LeakyReLU()(layer_16)

mcc_16 = Dense(32)(msk_15)
msk_16 = add([mcc_16, layer_16])

out_layer = Dense(1)(msk_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


In [33]:
%%time
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000
5/5 - 7s - loss: 53.0726 - mean_absolute_error: 53.0726 - val_loss: 1.8560 - val_mean_absolute_error: 1.8560 - 7s/epoch - 1s/step
Epoch 2/3000
5/5 - 0s - loss: 11.3848 - mean_absolute_error: 11.3848 - val_loss: 0.9766 - val_mean_absolute_error: 0.9766 - 148ms/epoch - 30ms/step
Epoch 3/3000
5/5 - 0s - loss: 5.4322 - mean_absolute_error: 5.4322 - val_loss: 6.7640 - val_mean_absolute_error: 6.7640 - 123ms/epoch - 25ms/step
Epoch 4/3000
5/5 - 0s - loss: 3.7512 - mean_absolute_error: 3.7512 - val_loss: 0.9288 - val_mean_absolute_error: 0.9288 - 152ms/epoch - 30ms/step
Epoch 5/3000
5/5 - 0s - loss: 1.9301 - mean_absolute_error: 1.9301 - val_loss: 1.3644 - val_mean_absolute_error: 1.3644 - 125ms/epoch - 25ms/step
Epoch 6/3000
5/5 - 0s - loss: 1.2068 - mean_absolute_error: 1.2068 - val_loss: 0.8080 - val_mean_absolute_error: 0.8080 - 151ms/epoch - 30ms/step
Epoch 7/3000
5/5 - 0s - loss: 0.9323 - mean_absolute_error: 0.9323 - val_loss: 0.7631 - val_mean_absolute_error: 0.7631 - 153

In [34]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

2/2 [==============================] - 0s 6ms/step
0.33035037267933715


In [35]:
import os
f=open('AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k[0])+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv.zip AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv'
os.system(cmd)


updating: AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv (deflated 51%)


0